# Assignment 2: Graphing Dino Fun World

### Assignment Description

Impressed by your previous work, the administrators of Dino Fun World have asked you to create some charts that they can use in their next presentation to upper management. The data used for this assignment will be the same as the data used for the previous assignment.

The administrators would like you to create four graphs:

Chart 1: A pie chart depicting visits to thrill ride attractions.

Chart 2: A bar chart depicting total visits to food stalls.

* Please query attractions with Category equal to "Food".

Chart 3: A line chart depicting attendance at the newest ride, Atmosfear over the course of the day.

* For this question, use data from the table "sequences". You can assume that all activity sequences are aligned by time (i.e., the first node of all sequences occurred at the same time) and are of the same length.

* You only need to consider the first 16 hours of records in the sequences, which is considered the open hours of the park in one day. Specifically, because an activity sequence from the "sequences" table was updated/recorded every 5 minutes, you need to extract the first 192 items (16h * 60m / 5m = 192).

* Your data list (which will be printed) must be in the format of tuple list (or list of lists). The first item in tuples is irrelevant, but you can put in some meaningful information. The second item is the count of visits at that moment. For example, your output should look like this (in Python syntax; not relevant to the correct answer): [ (0, 0), (1, 7), (2, 3), …, (190, 4), (191, 5) ].

Chart 4: A box-and-whisker plot depicting total visits to the park's Kiddie Rides.

* The to-be-printed, the  data list must be ordered by AttractionID (a field in the table "attraction").


### Directions

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkin', 'attractions', and 'sequences'. The database file is named 'dinofunworld.db' and is available in the read only directory of the Jupyter Notebook environment (i.e., readonly/dinofunworld.db). It can also be accessed by selecting File > Open > dinofunworld.db.

The information contained in each of these tables is listed below:

`checkin`:
    - The check-in data for all visitors for the day in the park. The data includes two types of check-ins: inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type
`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carousel, etc.
    - Fields: AttractionID, Name, Region, Category, type
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
Using the provided data, create the four visualizations that the administration requested: the pie chart, bar chart, line chart, and box-and-whisker plot.   

### Submission Directions for Assignment Deliverables

This assignment will be auto-graded. We recommend that you use Jupyter Notebook in your browser to complete and submit this assignment. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly.

Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed. (Graded Cell and PartID comments must be in the same line for proper execution of code.)

Please execute each cell in Jupyter Notebook before submitting.

If you choose to download the file and work on your assignment locally, you can also upload your file to each part in the programming assignment submission space. The file you submit should be named "Assignment_2.ipynb".

### Evaluation

There are four parts in the grading, and each part has one test case where the total number of points for all parts is 4. If some part of your data is incorrect, you will get a partial score of 0.25 or 0.50. If the submission fails, we will return the corresponding error messages. If the submission is correct, you will see "Correct" with 1.0 point for each part.

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
db_filename = 'dinofunworld.db'
#conn = sqlite3.connect("./readonly/{0}".format(db_filename)) 
conn = sqlite3.connect(db_filename) 
c = conn.cursor()
# db_filename = 'dinofunworld.db'
# conn = sqlite3.connect("./readonly/{0}".format(db_filename))
# c = conn.cursor()

In [ ]:
''' Queries the database for a specific category name in category 
    table and returns {name, attractionID} from each of the rows 
    satisfying condition. categoryName is filtered using %categoryName%'''
def getRideAttractionIDsAndNames(categoryName):
    return c.execute("SELECT Name, AttractionID FROM attraction WHERE Category like '%{0}%'".format(categoryName)).fetchall()

In [ ]:
''' Queries the database checkin table for specific categories 
    and returns {count of visitors to each attraction, attractionID}'''
def getVisitorCountToEachAttraction(attractionIdentifiers):
    innerQueryValue = ','.join(attractionIdentifiers)
    return c.execute("SELECT COUNT(*), attraction FROM checkin WHERE attraction IN ({0}) GROUP BY attraction".format(innerQueryValue)).fetchall()

In [ ]:
# Graded Cell, PartID: gtMqY
# Make a Pie Chart of the visits to Thrill Ride attractions. For this question,
#  display the pie chart in the notebook and print the data used to create the
#  pie chart as a list of lists (ex: [['Ride 1', 10], ['Ride 2', 100], ...])
thrillRideAttractionNames = getRideAttractionIDsAndNames('Thrill')
thrillRideAttractionDetailsDataFrame = pd.DataFrame.from_records(thrillRideAttractionNames, columns=['Name','AttractionID'])

# to use in the inner query while fetching visits
attractionIds = list(map(lambda x: str(x), thrillRideAttractionDetailsDataFrame['AttractionID']))

thrillRideAttractionsVisitors = getVisitorCountToEachAttraction(attractionIds)
thrillRideAttractionVisitorsDataFrame = pd.DataFrame.from_records(thrillRideAttractionsVisitors, columns=['Visits','AttractionID'])

# to have a single point of reference for attraction names and count of visits
dataDictionary = {}
for index in thrillRideAttractionDetailsDataFrame.index:
    entry = (thrillRideAttractionDetailsDataFrame['Name'][index], thrillRideAttractionVisitorsDataFrame['Visits'][index])
    dataDictionary.update([entry])
labelValues = []
printFormatList = []
for attractionName, visitorCount in dataDictionary.items():
    labelValues.append(attractionName+'('+str(visitorCount)+')')
    printFormatList.append([attractionName, visitorCount])

fig1, ax1 = plt.subplots()
patches, texts, autotexts = ax1.pie(dataDictionary.values(), labels=labelValues, autopct='%1.1f%%', startangle=90, shadow=False)
for text in texts:
    text.set_color('black')
for autotext in autotexts:
    autotext.set_color('black')
ax1.axis('equal')  
plt.tight_layout()
plt.title('Visits to Thrill Ride attractions')

plt.show()
print(printFormatList)

In [ ]:
# Graded Cell, PartID: 9Ocyl
# Make a bar chart of total visits to food stalls. For this question,
#  display the bar chart in the notebook and print the data used to create the
#  bar chart as a list of lists (ex: [['Stall 1', 10], ['Stall 2', 50], ...])
foodAttractionNames = getRideAttractionIDsAndNames('Food')
foodAttractionDetailsDataFrame = pd.DataFrame.from_records(foodAttractionNames, columns=['Name','AttractionID'])

# to use in the inner query while fetching visits
foodAttractionIds = list(map(lambda x: str(x), foodAttractionDetailsDataFrame['AttractionID']))
foodAttractionsVisitors = getVisitorCountToEachAttraction(foodAttractionIds)
foodAttractionVisitorsDataFrame = pd.DataFrame.from_records(foodAttractionsVisitors, columns=['Visits','AttractionID'])
dataDictionary = {}
for index in foodAttractionDetailsDataFrame.index:
    entry = (foodAttractionDetailsDataFrame['Name'][index], foodAttractionVisitorsDataFrame['Visits'][index])
    dataDictionary.update([entry])

plt.figure(figsize=(25, 5))
plt.bar(range(len(dataDictionary.values())), dataDictionary.values(), width = 0.2)
plt.xticks(range(len(dataDictionary.values())), dataDictionary.keys())
plt.ylabel('Number of Visits')
plt.title('Total visits to all foood stalls')
plt.show()

printFormatList = [[attractionName, visitorCount] for attractionName, visitorCount in dataDictionary.items()]
print(printFormatList)

In [ ]:
sequencesBoundaryValue = 192
def getAttendance(sequenceString):
    firstSixteenHrsSequences = sequenceString.split('-', sequencesBoundaryValue)[:-1]
    return list(map(lambda y: 1 if y==atmosFearAttractionId else 0, firstSixteenHrsSequences))

In [ ]:
# Graded Cell, PartID: 0zcEV
# Make a line chart of attendance at Atmosfear every five minutes. Again,
#  display the line chart in the notebook and print the data used to create the
#  chart as a list of lists (ex: [['Stall 1', 10], ['Stall 2', 50], ...])
rideName = 'Atmosfear'
atmosFearFromDB = c.execute("SELECT AttractionID from attraction WHERE Name='{0}'".format(rideName)).fetchall()[0][0]

# return type from db will be of type int
atmosFearAttractionId = str(atmosFearFromDB)

sequences = c.execute("SELECT Sequence FROM Sequences").fetchall()

sequenceDataFrame = pd.DataFrame.from_records(sequences, columns = ['Sequences'])
atmosFearForEveryVisitor = list(map(lambda x: getAttendance(x), sequenceDataFrame['Sequences']))

summedAtmosFearVisitorsValue = [0]*sequencesBoundaryValue

for atmosFearAVisitor in atmosFearForEveryVisitor:
    summedAtmosFearVisitorsValue = [sum(x) for x in zip(summedAtmosFearVisitorsValue, atmosFearAVisitor)]

fiveMinuteTimeSlot = [i for i in range(0,sequencesBoundaryValue)]

plt.plot(fiveMinuteTimeSlot, summedAtmosFearVisitorsValue)
plt.title('Attendance at Atmosfear')
plt.xlabel('5 minute time slot')
plt.ylabel('Number of visitors')
plt.grid(True)

#to show the legend/scale
plt.legend(['X-axis - 1 unit = 5 minutes'], loc=1)
f = plt.gcf()
fig_width, _ = f.get_size_inches()
f.set_figwidth(fig_width*2.5)

plt.show()
print(summedAtmosFearVisitorsValue)

In [ ]:
# Graded Cell, PartID: zdzaT
# Make a box plot of total visits to rides in the Kiddie Rides category. For
#  this question, display the box plot in the notebook and print the number of
#  visits to each ride as a list (ex: [3, 4, 5, 6, ...])
kiddieAttractionNames = getRideAttractionIDsAndNames('Kiddie')
kiddieAttractionDetailsDataFrame = pd.DataFrame.from_records(kiddieAttractionNames, columns=['Name','AttractionID'])

kiddieAttractionIds = list(map(lambda x: str(x), kiddieAttractionDetailsDataFrame['AttractionID']))
kiddieAttractionsVisitors = getVisitorCountToEachAttraction(kiddieAttractionIds)
kiddieAttractionVisitorsDataFrame = pd.DataFrame.from_records(kiddieAttractionsVisitors, columns=['Visits','AttractionID'])

plt.boxplot(x=kiddieAttractionVisitorsDataFrame['Visits'],vert=False)
plt.title('Total visits to rides in the Kiddie Rides')
plt.xlabel('Number of Visits')
plt.show()
print(list(kiddieAttractionVisitorsDataFrame['Visits']))